# Stratégie Envelope Multicoin - Mode Adaptatif par Régime

Ce notebook démontre l'utilisation du système de détection de régime adaptatif qui ajuste automatiquement :
- **Le mode de trading** (LONG_ONLY, LONG_SHORT, SHORT_ONLY)
- **Les paramètres d'enveloppe** (largeur, TP, SL, trailing)
- **La gestion des positions** (fermeture des positions interdites lors de changement de régime)

## Régimes détectés
- **BULL** : Tendance haussière confirmée (close > EMA200, EMA50 > EMA200, slope EMA200 ≥ 0)
- **BEAR** : Tendance baissière confirmée (close < EMA200, EMA50 < EMA200, slope EMA200 < 0)
- **RECOVERY** : Phase de transition ou consolidation

## Mapping Régime → Mode
- BULL → LONG_ONLY (pas de shorts)
- RECOVERY → LONG_ONLY (attente de confirmation)
- BEAR → LONG_SHORT (ou SHORT_ONLY si simplified=True)

In [ ]:
# Configuration et imports
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

# Imports core régime adaptatif
from core import (
    Regime,
    Mode,
    calculate_regime_series,
    get_mode_for_regime,
    DEFAULT_PARAMS,
    handle_regime_change
)

# Imports utilitaires
from utilities.data_manager import ExchangeDataManager

print("✅ Modules chargés avec succès")
print("📊 Régimes disponibles:", [r.value for r in Regime])
print("🎯 Modes disponibles:", [m.value for m in Mode])

In [ ]:
# Chargement des données BTC (proxy pour régime global)
exchange = ExchangeDataManager(exchange_name="binance", path_download="../../database/exchanges")

# Charger BTC pour détection de régime
df_btc = exchange.load_data("BTC/USDT:USDT", "1h")
df_btc = df_btc.loc["2020-01-01":]

# Calculer EMA50 et EMA200 pour détection de régime
df_btc['ema50'] = df_btc['close'].ewm(span=50, adjust=False).mean()
df_btc['ema200'] = df_btc['close'].ewm(span=200, adjust=False).mean()

print(f"✅ Données BTC chargées: {len(df_btc)} barres")
print(f"   Période: {df_btc.index[0]} → {df_btc.index[-1]}")
print(f"\nPremières lignes:")
df_btc[['close', 'ema50', 'ema200']].head()

In [ ]:
# Détection de régime global (proxy BTC)
confirm_n = 12  # Hystérésis : 12 barres pour confirmer changement

regimes = calculate_regime_series(
    df_btc=df_btc[['close', 'ema50', 'ema200']],
    confirm_n=confirm_n
)

# Ajouter au dataframe principal
df_btc['regime'] = regimes

# Analyse de la distribution des régimes
print("="*80)
print("DISTRIBUTION DES RÉGIMES (BTC proxy global)")
print("="*80)

regime_counts = regimes.value_counts()
for regime, count in regime_counts.items():
    pct = count / len(regimes) * 100
    print(f"{regime.value.upper():10s}: {count:5d} barres ({pct:5.1f}%)")

print(f"\nTotal: {len(regimes)} barres")
print(f"Hystérésis appliquée: {confirm_n} barres")

# Afficher échantillon avec régimes
print("\nÉchantillon (2024):")
df_btc.loc['2024', ['close', 'ema50', 'ema200', 'regime']].head(10)

In [ ]:
# Créer un DataFrame de décisions avec mode et paramètres par date
decisions = []

for date, regime in regimes.items():
    mode = get_mode_for_regime(regime, simplified=False)
    params = DEFAULT_PARAMS[regime]
    
    decisions.append({
        'date': date,
        'regime': regime.value,
        'mode': mode.value,
        'envelope_std': params.envelope_std,
        'tp_mult': params.tp_mult,
        'sl_mult': params.sl_mult,
        'trailing': params.trailing,
        'allow_shorts': params.allow_shorts
    })

df_decisions = pd.DataFrame(decisions)
df_decisions.set_index('date', inplace=True)

print("="*80)
print("DÉCISIONS PAR RÉGIME")
print("="*80)
print(df_decisions.groupby('regime').first())

# Compter les transitions
transitions = (df_decisions['regime'] != df_decisions['regime'].shift()).sum()
print(f"\nNombre de transitions de régime: {transitions}")
print(f"Durée moyenne d'un régime: {len(df_decisions) / transitions:.1f} barres")

In [ ]:
# Visualisation : Régimes dans le temps avec prix BTC
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer sous-graphiques
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=('Prix BTC avec EMAs', 'Régime détecté')
)

# Prix et EMAs
fig.add_trace(
    go.Scatter(x=df_btc.index, y=df_btc['close'], name='BTC Close', line=dict(color='white', width=1)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df_btc.index, y=df_btc['ema50'], name='EMA50', line=dict(color='orange', width=1)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df_btc.index, y=df_btc['ema200'], name='EMA200', line=dict(color='blue', width=1)),
    row=1, col=1
)

# Régimes (codés en nombre pour affichage)
regime_map = {Regime.BULL: 2, Regime.RECOVERY: 1, Regime.BEAR: 0}
df_btc['regime_num'] = df_btc['regime'].map(regime_map)

colors = {'bull': 'green', 'recovery': 'yellow', 'bear': 'red'}
for regime in [Regime.BULL, Regime.RECOVERY, Regime.BEAR]:
    mask = df_btc['regime'] == regime
    fig.add_trace(
        go.Scatter(
            x=df_btc.index[mask],
            y=df_btc['regime_num'][mask],
            name=regime.value.upper(),
            mode='markers',
            marker=dict(color=colors[regime.value], size=3)
        ),
        row=2, col=1
    )

fig.update_layout(
    title='Détection de Régime - BTC Proxy Global',
    height=800,
    template='plotly_dark'
)

fig.update_yaxes(title_text="Prix ($)", row=1, col=1, type="log")
fig.update_yaxes(title_text="Régime", row=2, col=1, tickvals=[0, 1, 2], ticktext=['BEAR', 'RECOVERY', 'BULL'])

fig.show()

## Analyse des Transitions de Régime

Cette cellule analyse les moments où le régime change et les positions qui devraient être fermées.

In [ ]:
# Détecter les transitions
df_decisions['regime_prev'] = df_decisions['regime'].shift(1)
transitions = df_decisions[df_decisions['regime'] != df_decisions['regime_prev']].copy()

print("="*80)
print("TRANSITIONS DE RÉGIME")
print("="*80)

if len(transitions) > 0:
    print(f"Nombre total de transitions: {len(transitions)}\n")
    
    for idx, row in transitions.head(10).iterrows():
        print(f"Date: {idx}")
        print(f"  {row['regime_prev']} → {row['regime']}")
        print(f"  Mode: {get_mode_for_regime(Regime(row['regime_prev']), False).value} → {row['mode']}")
        
        # Simuler positions ouvertes (exemple)
        if row['regime'] == 'bull':
            print(f"  Action: Fermer toutes les positions SHORT")
        elif row['mode'] == 'SHORT_ONLY':
            print(f"  Action: Fermer toutes les positions LONG")
        else:
            print(f"  Action: Aucune fermeture (mode LONG_SHORT)")
        
        print(f"  Nouveaux params: envelope_std={row['envelope_std']}, tp={row['tp_mult']}, sl={row['sl_mult']}")
        print()
else:
    print("Aucune transition détectée (régime stable)")

## Paramètres par Régime

Affichage des paramètres DEFAULT_PARAMS pour chaque régime.

In [ ]:
# Afficher les paramètres par régime
import pandas as pd
from dataclasses import asdict

params_df = pd.DataFrame([asdict(DEFAULT_PARAMS[r]) for r in Regime], index=[r.value.upper() for r in Regime])

print("="*80)
print("PARAMÈTRES PAR RÉGIME")
print("="*80)
print(params_df.to_string())

print("\n" + "="*80)
print("OBSERVATIONS CLÉS")
print("="*80)
print("1. BULL: Enveloppes larges (0.12), stops larges, pas de shorts")
print("2. RECOVERY: Enveloppes moyennes (0.10), stops serrés, shorts autorisés")
print("3. BEAR: Enveloppes serrées (0.07), TP/SL courts, shorts autorisés")
print("\nLa largeur d'envelope diminue du bull au bear pour s'adapter à la volatilité.")

## Export des décisions pour intégration backtest

Sauvegarde du DataFrame de décisions pour utilisation dans le backtest complet.

In [ ]:
# Sauvegarder les décisions
output_path = "../../backtests/regime_decisions.csv"
df_decisions.to_csv(output_path)

print(f"✅ Décisions exportées vers: {output_path}")
print(f"   {len(df_decisions)} lignes, {len(df_decisions.columns)} colonnes")
print(f"\nPremières lignes:")
print(df_decisions.head())

# Résumé final
print("\n" + "="*80)
print("RÉSUMÉ")
print("="*80)
print(f"Période analysée: {df_decisions.index[0]} → {df_decisions.index[-1]}")
print(f"Régimes détectés: {', '.join(df_decisions['regime'].unique())}")
print(f"Modes utilisés: {', '.join(df_decisions['mode'].unique())}")
print(f"Transitions: {len(transitions)}")
print("\nProchaine étape: Intégrer ces décisions dans le backtest EnvelopeMulti_v2")